# Toxrefdb - Toxicity Reference Database
Example notebook shows how to generate the [toxrefdb blog post](https://insilica.co/posts/2023-11-11-toxrefdb/)

In [3]:
import sqlite3, pandas as pd
import biobricks as bb

# Access the ToxRefDB asset
assets = bb.assets("toxrefdb")
conn = sqlite3.connect(assets.toxrefdb_sqlite)

# Query to get NOAEL data
query = """
SELECT c.preferred_name AS chemical_name,
       g.name AS guideline_name,
       p.dose_level < p.max_dose_level AS adverse
FROM pod p
JOIN chemical c ON p.chemical_id = c.chemical_id
JOIN study s ON p.study_id = s.study_id
JOIN guideline g ON s.guideline_id = g.guideline_id
WHERE p.pod_type = 'noael'
"""
data = pd.read_sql_query(query, conn)

# Count hazardous and non-hazardous compounds for each guideline
summary = data.groupby(['guideline_name', 'adverse']).size().unstack(fill_value=0)
summary


TypeError: 'types.SimpleNamespace' object is not subscriptable

In [ ]:
import matplotlib.pyplot as plt
# Plot the data
summary.plot(kind='bar', stacked=True)
plt.xlabel('Guideline')
plt.ylabel('Count')
plt.title('Counts of Hazardous and Non-Hazardous Compounds by Guideline')
plt.legend(['Non-Hazardous', 'Hazardous'], loc='upper right')
plt.tight_layout()
plt.show()